Возьмите датасет www.kaggle.com...ta/kernels

1. Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)  
2. Обучите на нем модели ResNet 18 и VGG 16 с использованием Fine-tuning (5-10 эпох)  
3. Добавьте аугментацию данных к пункту 2  

Сравните качество всех 3 полученных подходов

### Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)

In [10]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [15]:
BATCH_SIZE = 64

In [4]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

In [5]:
train_dataset = tv.datasets.ImageFolder("hymenoptera_data/train", transform=transoforms)
test_dataset = tv.datasets.ImageFolder("hymenoptera_data/val", transform=transoforms)

train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [24]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [25]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

#### ResNet 18

In [8]:
resnet = tv.models.resnet18(pretrained=False)

E:\DS\anaconda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\DS\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
resnet.fc = nn.Linear(in_features=512, out_features=2)

In [11]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(resnet.parameters(), lr=lr)
train(resnet, train_iter, test_iter, trainer, num_epochs)

--------------------
epoch 1, loss 1.5944, train acc 0.275, test acc 0.458, time 6.0 sec
--------------------
epoch 2, loss 4.5155, train acc 0.980, test acc 0.542, time 3.4 sec
--------------------
epoch 3, loss 16.7849, train acc 0.496, test acc 0.542, time 3.4 sec
--------------------
epoch 4, loss 3.4331, train acc 0.020, test acc 0.458, time 3.4 sec
--------------------
epoch 5, loss 1.0391, train acc 0.504, test acc 0.542, time 3.4 sec
--------------------
epoch 6, loss 0.7186, train acc 0.234, test acc 0.542, time 3.4 sec
--------------------
epoch 7, loss 0.7424, train acc 0.496, test acc 0.542, time 3.4 sec
--------------------
epoch 8, loss 0.7128, train acc 0.496, test acc 0.542, time 3.4 sec
--------------------
epoch 9, loss 0.7094, train acc 0.496, test acc 0.542, time 3.4 sec
--------------------
epoch 10, loss 0.7049, train acc 0.496, test acc 0.542, time 3.4 sec


In [47]:
model.eval()
torch.cuda.empty_cache()

#### VGG 16

In [12]:
vgg = tv.models.vgg16(pretrained = False)

In [13]:
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [34]:
vgg.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [15]:
trainer = torch.optim.Adam(vgg.parameters(), lr=lr)
train(vgg, train_iter, test_iter, trainer, num_epochs)

--------------------
epoch 1, loss 3.8541, train acc 0.357, test acc 0.542, time 17.2 sec
--------------------
epoch 2, loss 0.7137, train acc 0.020, test acc 0.458, time 14.9 sec
--------------------
epoch 3, loss 0.6941, train acc 0.504, test acc 0.458, time 15.1 sec
--------------------
epoch 4, loss 0.6938, train acc 0.504, test acc 0.458, time 15.1 sec
--------------------
epoch 5, loss 0.6937, train acc 0.504, test acc 0.458, time 15.6 sec
--------------------
epoch 6, loss 0.6937, train acc 0.504, test acc 0.458, time 15.7 sec
--------------------
epoch 7, loss 0.6937, train acc 0.504, test acc 0.458, time 15.9 sec
--------------------
epoch 8, loss 0.6937, train acc 0.504, test acc 0.458, time 15.2 sec
--------------------
epoch 9, loss 0.6937, train acc 0.504, test acc 0.458, time 15.3 sec
--------------------
epoch 10, loss 0.6937, train acc 0.504, test acc 0.458, time 14.6 sec


### Обучите на нем модели ResNet 18 и VGG 16 с использованием Fine-tuning (5-10 эпох)

#### ResNet 18 Trained

In [18]:
resnet_train = tv.models.resnet18(pretrained=True)

In [20]:
for param in resnet_train.parameters():
    param.requires_grad = False

In [21]:
resnet_train.fc = nn.Linear(in_features=512, out_features=2)

In [23]:
print("Params to learn:")
params_to_update = []
for name, param in resnet_train.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [27]:
trainer = torch.optim.Adam(params_to_update, lr=lr)
train(resnet_train, train_iter, test_iter, trainer, num_epochs)

--------------------
epoch 1, loss 1.1209, train acc 0.238, test acc 0.477, time 2.8 sec
--------------------
epoch 2, loss 0.6624, train acc 0.566, test acc 0.634, time 2.8 sec
--------------------
epoch 3, loss 0.4331, train acc 0.836, test acc 0.739, time 3.1 sec
--------------------
epoch 4, loss 0.4852, train acc 0.779, test acc 0.712, time 3.0 sec
--------------------
epoch 5, loss 0.5519, train acc 0.697, test acc 0.725, time 2.8 sec
--------------------
epoch 6, loss 0.5264, train acc 0.758, test acc 0.791, time 2.8 sec
--------------------
epoch 7, loss 0.4867, train acc 0.816, test acc 0.797, time 2.9 sec
--------------------
epoch 8, loss 0.4570, train acc 0.811, test acc 0.797, time 2.8 sec
--------------------
epoch 9, loss 0.4230, train acc 0.836, test acc 0.804, time 2.7 sec
--------------------
epoch 10, loss 0.3966, train acc 0.869, test acc 0.824, time 2.8 sec


#### VGG 16 Trained

In [28]:
vgg_train = tv.models.vgg16(pretrained=True)

E:\DS\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\cefeu/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [29]:
for param in vgg_train.parameters():
    param.requires_grad = False

In [31]:
vgg_train.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [32]:
print("Params to learn:")
params_to_update = []
for name, param in vgg_train.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [33]:
trainer = torch.optim.Adam(params_to_update, lr=lr)
train(vgg_train, train_iter, test_iter, trainer, num_epochs)

--------------------
epoch 1, loss 1.7638, train acc 0.357, test acc 0.458, time 3.9 sec
--------------------
epoch 2, loss 0.4782, train acc 0.721, test acc 0.856, time 3.9 sec
--------------------
epoch 3, loss 0.4605, train acc 0.762, test acc 0.712, time 4.0 sec
--------------------
epoch 4, loss 0.5529, train acc 0.680, test acc 0.863, time 4.0 sec
--------------------
epoch 5, loss 0.4169, train acc 0.828, test acc 0.843, time 4.0 sec
--------------------
epoch 6, loss 0.3487, train acc 0.861, test acc 0.771, time 4.0 sec
--------------------
epoch 7, loss 0.3092, train acc 0.873, test acc 0.804, time 4.0 sec
--------------------
epoch 8, loss 0.2589, train acc 0.914, test acc 0.843, time 4.0 sec
--------------------
epoch 9, loss 0.2326, train acc 0.922, test acc 0.863, time 4.1 sec
--------------------
epoch 10, loss 0.2307, train acc 0.922, test acc 0.869, time 4.1 sec


### Добавьте аугментацию данных к пункту 2

In [12]:
from torchvision import transforms

In [28]:
transformations = transforms.Compose([
    transforms.Resize(size=(255, 255), ),
    transforms.RandomRotation(degrees=90),
    transforms.RandomVerticalFlip(),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

In [29]:
train_dataset = tv.datasets.ImageFolder("hymenoptera_data/train", transform=transformations)
test_dataset = tv.datasets.ImageFolder("hymenoptera_data/val", transform=transformations)

train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

#### VGG 16 с аугментацией данных

In [30]:
vgg_aug = tv.models.vgg16(pretrained=True)

In [31]:
for param in vgg_aug.parameters():
    param.requires_grad = False

In [32]:
vgg_aug.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [33]:
print("Params to learn:")
params_to_update = []
for name, param in vgg_aug.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [34]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train(vgg_aug, train_iter, test_iter, trainer, 10)

--------------------
epoch 1, loss 1.8622, train acc 0.365, test acc 0.458, time 3.8 sec
--------------------
epoch 2, loss 0.3728, train acc 0.852, test acc 0.824, time 3.9 sec
--------------------
epoch 3, loss 0.4239, train acc 0.746, test acc 0.771, time 3.9 sec
--------------------
epoch 4, loss 0.4526, train acc 0.746, test acc 0.863, time 3.8 sec
--------------------
epoch 5, loss 0.3146, train acc 0.861, test acc 0.922, time 3.8 sec
--------------------
epoch 6, loss 0.2582, train acc 0.889, test acc 0.882, time 3.8 sec
--------------------
epoch 7, loss 0.2470, train acc 0.910, test acc 0.882, time 4.1 sec
--------------------
epoch 8, loss 0.1967, train acc 0.934, test acc 0.889, time 3.8 sec
--------------------
epoch 9, loss 0.1634, train acc 0.939, test acc 0.915, time 3.9 sec
--------------------
epoch 10, loss 0.1615, train acc 0.934, test acc 0.941, time 4.0 sec


#### ResNet 18 с аугментацией данных

In [35]:
resnet_aug = tv.models.resnet18(pretrained=True)

E:\DS\anaconda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
for param in resnet_aug.parameters():
    param.requires_grad = False

In [37]:
resnet_aug.fc = nn.Linear(in_features=512, out_features=2)

In [38]:
print("Params to learn:")
params_to_update = []
for name, param in resnet_aug.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [39]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train(resnet_aug, train_iter, test_iter, trainer, 10)

--------------------
epoch 1, loss 1.1766, train acc 0.373, test acc 0.464, time 2.9 sec
--------------------
epoch 2, loss 0.7307, train acc 0.553, test acc 0.529, time 2.7 sec
--------------------
epoch 3, loss 0.6542, train acc 0.627, test acc 0.562, time 2.8 sec
--------------------
epoch 4, loss 0.6854, train acc 0.545, test acc 0.608, time 2.7 sec
--------------------
epoch 5, loss 0.6921, train acc 0.500, test acc 0.641, time 2.8 sec
--------------------
epoch 6, loss 0.6672, train acc 0.545, test acc 0.686, time 2.7 sec
--------------------
epoch 7, loss 0.5958, train acc 0.730, test acc 0.745, time 2.7 sec
--------------------
epoch 8, loss 0.5197, train acc 0.762, test acc 0.810, time 2.6 sec
--------------------
epoch 9, loss 0.5041, train acc 0.816, test acc 0.817, time 2.7 sec
--------------------
epoch 10, loss 0.4901, train acc 0.824, test acc 0.824, time 2.9 sec


#### Наилучшим, из трех подходов, качеством обладает модель предобученной VGG 16 c аугментацией данных.